# Baseline characteristics table - part 1*

### * This code belongs to the paper "Early Prediction of End Stage Kidney Disease Based on Cumulative Estimated Glomerular Filtration Rate Velocity"

In [ ]:
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime

In [ ]:
diagnosis_data = pd.read_csv("DIAGNOSIS_2.csv")
datapool_ESRD = pd.read_csv('Final_ESRD_group_done_pandas.csv')
full_date_dataset = pd.read_csv('merged_dataset_dates_timedeltas_full.csv')


datapool_ESRD = pd.read_csv('Final_ESRD_group_done_pandas.csv')
datapool_ESRD = datapool_ESRD.drop(columns=datapool_ESRD.columns[0])
datapool_ESRD = datapool_ESRD.drop_duplicates()
datapool_ESRD_dropped = datapool_ESRD.groupby('patient_sk').agg({'eGFR_EPI' : 'count'}).reset_index()[datapool_ESRD.groupby('patient_sk').agg({'eGFR_EPI' : 'count'}).reset_index().eGFR_EPI >=9]
datapool_ESRD_dropped = datapool_ESRD_dropped.drop('eGFR_EPI', axis =1)
datapool_ESRD = datapool_ESRD.merge(datapool_ESRD_dropped, on = 'patient_sk', how = 'inner')

datapool_ESRD['Date'] = pd.to_datetime(datapool_ESRD['Date'])
datapool_ESRD['Date'] = pd.to_datetime(datapool_ESRD['Date'])
datapool_ESRD_dates = datapool_ESRD.groupby('patient_sk').agg({'Date': 'min'})
datapool_ESRD_dates = datapool_ESRD_dates.reset_index()
datapool_ESRD = datapool_ESRD.merge(datapool_ESRD_dates, on = 'patient_sk', how='left')
datapool_ESRD['Date_seconds'] = datapool_ESRD['Date_x'] - datapool_ESRD['Date_y']
datapool_ESRD = datapool_ESRD.rename({'Date_x':'Date'}, axis = 1)
datapool_ESRD = datapool_ESRD.drop('Date_y', axis = 1)
datapool_ESRD['Date_seconds'] = datapool_ESRD['Date_seconds'].dt.total_seconds()

datapool_ESRD_patients = datapool_ESRD.groupby('patient_sk').agg({'Date': 'min'})
datapool_ESRD_patients = datapool_ESRD_patients.reset_index()

datapool_ESRD_patients_eGFR = datapool_ESRD.merge(datapool_ESRD_patients, on=['patient_sk', 'Date'], how ='inner')
datapool_ESRD_patients_eGFR = datapool_ESRD_patients_eGFR.drop_duplicates('patient_sk')
datapool_ESRD_patients_eGFR = datapool_ESRD_patients_eGFR[datapool_ESRD_patients_eGFR['eGFR_EPI']>=60]

datapool_ESRD_new = datapool_ESRD.merge(datapool_ESRD_patients_eGFR['patient_sk'], on = 'patient_sk', how = 'inner')

datapool_ESRD = datapool_ESRD_new
datapool_ESRD = datapool_ESRD.drop_duplicates()


In [ ]:
merged_dataset = full_date_dataset.merge(diagnosis_data, on = 'patient_sk' , how = 'inner')
merged_dataset = merged_dataset.rename(columns={'eGFR_EPI': 'Trigger_eGFR'})

In [ ]:
## Replacing NaN and NaT (pd.null) by 0
pp = []
for i in range(len(list(merged_dataset['patient_sk']))):
   
    if np.isnan(merged_dataset.New_label[i]) == False:
        pp.append(merged_dataset.patient_sk[0])
        

trigger_below_20 = list(np.array(list(merged_dataset['patient_sk']))[np.array(list(merged_dataset['Trigger_eGFR']))<20])
print('trigger_below_20:', len(trigger_below_20))

a = np.array(list(merged_dataset['Trigger_eGFR']))<40 
b = np.array(list(merged_dataset['Trigger_eGFR']))>=20
pa = np.array(list(merged_dataset['patient_sk']))[a]
pb = np.array(list(merged_dataset['patient_sk']))[b]                  
trigger_between_20_40 = list(np.intersect1d(pb, pa))
print('trigger_between_20_40:', len(trigger_between_20_40))

a = np.array(list(merged_dataset['Trigger_eGFR']))<60 
b = np.array(list(merged_dataset['Trigger_eGFR']))>=40
pa = np.array(list(merged_dataset['patient_sk']))[a]
pb = np.array(list(merged_dataset['patient_sk']))[b]                  
trigger_between_40_60 = list(np.intersect1d(pb, pa))
print('trigger_between_40_60:', len(trigger_between_40_60))

trigger_above_60 = np.array(list(merged_dataset['patient_sk']))[np.array(list(merged_dataset['Trigger_eGFR']))>=60]
print('trigger_above_60:', len(trigger_above_60))

no_trigger = list(set(list(merged_dataset['patient_sk'])) - (set(trigger_below_20)) - set(trigger_between_20_40) - set(trigger_between_40_60) - set(trigger_above_60))
print('no_trigger:', len(no_trigger))

## Diagnosis data

In [ ]:
# Having dialysis AFTER trigger date

merged_dataset['Diagnosis_admission_date_dialysis'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_dialysis'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_dialysis = list(merged_dataset['Diagnosis_admission_date_dialysis'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_dialysis)):
    if pd.isnull(Diagnosis_admission_date_dialysis[i]):
        Diagnosis_admission_date_dialysis[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_dialysis)):
    if Diagnosis_admission_date_dialysis[i] >= Trigger_date[i] and Diagnosis_admission_date_dialysis[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_dialysis: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))

merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])
count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_dialysis)):
    if Diagnosis_admission_date_dialysis[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_dialysis[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

In [ ]:
# Having transplant AFTER trigger date

merged_dataset['Diagnosis_admission_date_transplant'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_transplant'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_transplant = list(merged_dataset['Diagnosis_admission_date_transplant'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_transplant)):
    if pd.isnull(Diagnosis_admission_date_transplant[i]):
        Diagnosis_admission_date_transplant[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_transplant)):
    if Diagnosis_admission_date_transplant[i] >= Trigger_date[i] and Diagnosis_admission_date_transplant[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_transplant: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))

merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])
count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_transplant)):
    if Diagnosis_admission_date_transplant[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_transplant[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

In [ ]:
merged_dataset['Diagnosis_admission_date_Nicotine_Tobacco_dependency'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Nicotine_Tobacco_dependency'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Nicotine_Tobacco_dependency = list(merged_dataset['Diagnosis_admission_date_Nicotine_Tobacco_dependency'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Nicotine_Tobacco_dependency)):
    if pd.isnull(Diagnosis_admission_date_Nicotine_Tobacco_dependency[i]):
        Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Nicotine_Tobacco_dependency)):
    if Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] <= Trigger_date[i] and Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Nicotine_Tobacco_dependency: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])
count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Nicotine_Tobacco_dependency)):
    if Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Nicotine_Tobacco_dependency <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Nicotine_Tobacco_dependency) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Hypertension'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Hypertension'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Hypertension = list(merged_dataset['Diagnosis_admission_date_Hypertension'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Hypertension)):
    if pd.isnull(Diagnosis_admission_date_Hypertension[i]):
        Diagnosis_admission_date_Hypertension[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Hypertension)):
    if Diagnosis_admission_date_Hypertension[i] <= Trigger_date[i] and Diagnosis_admission_date_Hypertension[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Hypertension: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Hypertension)):
    if Diagnosis_admission_date_Hypertension[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Hypertension[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Hypertension <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Hypertension) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Diabetes'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Diabetes'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Diabetes = list(merged_dataset['Diagnosis_admission_date_Diabetes'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Diabetes)):
    if pd.isnull(Diagnosis_admission_date_Diabetes[i]):
        Diagnosis_admission_date_Diabetes[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Diabetes)):
    if Diagnosis_admission_date_Diabetes[i] <= Trigger_date[i] and Diagnosis_admission_date_Diabetes[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Diabetes: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Diabetes)):
    if Diagnosis_admission_date_Diabetes[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Diabetes[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Diabetes <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Diabetes) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Coronary_Artery_Disease'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Coronary_Artery_Disease'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Coronary_Artery_Disease = list(merged_dataset['Diagnosis_admission_date_Coronary_Artery_Disease'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Coronary_Artery_Disease)):
    if pd.isnull(Diagnosis_admission_date_Coronary_Artery_Disease[i]):
        Diagnosis_admission_date_Coronary_Artery_Disease[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Coronary_Artery_Disease)):
    if Diagnosis_admission_date_Coronary_Artery_Disease[i] <= Trigger_date[i] and Diagnosis_admission_date_Coronary_Artery_Disease[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Coronary_Artery_Disease: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Coronary_Artery_Disease)):
    if Diagnosis_admission_date_Coronary_Artery_Disease[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Coronary_Artery_Disease[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Coronary_Artery_Disease <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Coronary_Artery_Disease) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Cerebrovascular_Disease'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Cerebrovascular_Disease'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Cerebrovascular_Disease = list(merged_dataset['Diagnosis_admission_date_Cerebrovascular_Disease'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Cerebrovascular_Disease)):
    if pd.isnull(Diagnosis_admission_date_Cerebrovascular_Disease[i]):
        Diagnosis_admission_date_Cerebrovascular_Disease[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Cerebrovascular_Disease)):
    if Diagnosis_admission_date_Cerebrovascular_Disease[i] <= Trigger_date[i] and Diagnosis_admission_date_Cerebrovascular_Disease[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Cerebrovascular_Disease: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Cerebrovascular_Disease)):
    if Diagnosis_admission_date_Cerebrovascular_Disease[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Cerebrovascular_Disease[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Cerebrovascular_Disease <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Cerebrovascular_Disease) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Peripheral_Vascular_Disease'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Peripheral_Vascular_Disease'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Peripheral_Vascular_Disease = list(merged_dataset['Diagnosis_admission_date_Peripheral_Vascular_Disease'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Peripheral_Vascular_Disease)):
    if pd.isnull(Diagnosis_admission_date_Peripheral_Vascular_Disease[i]):
        Diagnosis_admission_date_Peripheral_Vascular_Disease[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Peripheral_Vascular_Disease)):
    if Diagnosis_admission_date_Peripheral_Vascular_Disease[i] <= Trigger_date[i] and Diagnosis_admission_date_Peripheral_Vascular_Disease[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Peripheral_Vascular_Disease: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Peripheral_Vascular_Disease)):
    if Diagnosis_admission_date_Peripheral_Vascular_Disease[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Peripheral_Vascular_Disease[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Peripheral_Vascular_Disease <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Peripheral_Vascular_Disease) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Sickle_Cell_Trait'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Sickle_Cell_Trait'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Sickle_Cell_Trait = list(merged_dataset['Diagnosis_admission_date_Sickle_Cell_Trait'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Sickle_Cell_Trait)):
    if pd.isnull(Diagnosis_admission_date_Sickle_Cell_Trait[i]):
        Diagnosis_admission_date_Sickle_Cell_Trait[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Sickle_Cell_Trait)):
    if Diagnosis_admission_date_Sickle_Cell_Trait[i] <= Trigger_date[i] and Diagnosis_admission_date_Sickle_Cell_Trait[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Sickle_Cell_Trait: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))

merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Sickle_Cell_Trait)):
    if Diagnosis_admission_date_Sickle_Cell_Trait[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Sickle_Cell_Trait[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))

merged_dataset[(merged_dataset.Diagnosis_admission_date_Sickle_Cell_Trait <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Sickle_Cell_Trait) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Hx_of_Cancer'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Hx_of_Cancer'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Hx_of_Cancer = list(merged_dataset['Diagnosis_admission_date_Hx_of_Cancer'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Hx_of_Cancer)):
    if pd.isnull(Diagnosis_admission_date_Hx_of_Cancer[i]):
        Diagnosis_admission_date_Hx_of_Cancer[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Hx_of_Cancer)):
    if Diagnosis_admission_date_Hx_of_Cancer[i] <= Trigger_date[i] and Diagnosis_admission_date_Hx_of_Cancer[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Hx_of_Cancer: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Hx_of_Cancer)):
    if Diagnosis_admission_date_Hx_of_Cancer[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Hx_of_Cancer[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))


merged_dataset[(merged_dataset.Diagnosis_admission_date_Hx_of_Cancer <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Hx_of_Cancer) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_Hypercholesterolemia'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Hypercholesterolemia'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Hypercholesterolemia = list(merged_dataset['Diagnosis_admission_date_Hypercholesterolemia'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Hypercholesterolemia)):
    if pd.isnull(Diagnosis_admission_date_Hypercholesterolemia[i]):
        Diagnosis_admission_date_Hypercholesterolemia[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Hypercholesterolemia)):
    if Diagnosis_admission_date_Hypercholesterolemia[i] <= Trigger_date[i] and Diagnosis_admission_date_Hypercholesterolemia[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Hypercholesterolemia: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Hypercholesterolemia)):
    if Diagnosis_admission_date_Hypercholesterolemia[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_Hypercholesterolemia[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))


merged_dataset[(merged_dataset.Diagnosis_admission_date_Hypercholesterolemia <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_Hypercholesterolemia) == False)].shape[0]

In [ ]:
merged_dataset['Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities = list(merged_dataset['Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities)):
    if pd.isnull(Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities[i]):
        Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities)):
    if Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities[i] <= Trigger_date[i] and Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))


merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'], errors='coerce')
Diagnosis_admission_date_ESRD = list(merged_dataset['Diagnosis_admission_date_ESRD'])

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities)):
    if Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities[i] >= Diagnosis_admission_date_ESRD[i] and Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
print('no_trigger: ',len(list(set(count_list) & set(no_trigger))))


merged_dataset[(merged_dataset.Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities <= merged_dataset.Diagnosis_admission_date_ESRD) & (np.isnan(merged_dataset.Diagnosis_admission_date_History_of_Urinary_Tract_Abnormalities) == False)].shape[0]

## Demographics

In [ ]:
data_no_trigger = datapool_ESRD.merge(pd.DataFrame({'patient_sk' : no_trigger}), on = 'patient_sk', how = 'inner')

In [ ]:
patients_list_ESRD = list(set(np.unique(list(datapool_ESRD['patient_sk']))))

dataaa_ESRD = []
for patient in patients_list_ESRD:
    data = datapool_ESRD.loc[datapool_ESRD['patient_sk'] == patient]
    data = data.sort_values(by=['Date'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)    
    dataaa_ESRD.append(data)
print(len(dataaa_ESRD))


In [ ]:
Ages = []
Genders = []
Races = []

i = 0
count_list = []
for patient in dataaa_ESRD:
    
    m_table = merged_dataset[merged_dataset.patient_sk == patient.patient_sk[0]]
    if np.isnan(m_table.New_label).values[0] == False:
        
        patient.Date = pd.to_datetime(patient.Date)
        ppp_date = pd.to_datetime(merged_dataset[merged_dataset.patient_sk == patient.patient_sk[0]].Trigger_date.values)

        Trigger_index = list(patient.Date).index(ppp_date)

        if pd.isnull(Trigger_index) == False:
            Ages.append(patient.Age[int(Trigger_index)])
            Genders.append(patient.Gender[int(Trigger_index)])
            Races.append(patient.Race[int(Trigger_index)])
            count_list.append(patient.patient_sk[0])
        i = i + 1
    

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

print(len(trigger_below_20), len(trigger_between_20_40), len(trigger_between_40_60), len(trigger_above_60))

In [ ]:
#Ages

trigger_below_20_age_mean = 0
trigger_between_20_40_age_mean = 0
trigger_between_40_60_age_mean = 0
trigger_above_60_age_mean = 0

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        trigger_below_20_age_mean = trigger_below_20_age_mean + (Ages[count_list.index(patient_sk)])/len(trigger_below_20)
    elif patient_sk in trigger_between_20_40:
        trigger_between_20_40_age_mean = trigger_between_20_40_age_mean + (Ages[count_list.index(patient_sk)])/len(trigger_between_20_40)    
    elif patient_sk in trigger_between_40_60:
        trigger_between_40_60_age_mean = trigger_between_40_60_age_mean + (Ages[count_list.index(patient_sk)])/len(trigger_between_40_60)
    elif patient_sk in trigger_above_60:
        trigger_above_60_age_mean = trigger_above_60_age_mean + (Ages[count_list.index(patient_sk)])/len(trigger_above_60)
        
print('trigger_below_20_age_mean = ', trigger_below_20_age_mean)
print('trigger_between_20_40_age_mean = ', trigger_between_20_40_age_mean)
print('trigger_between_40_60_age_mean = ', trigger_between_40_60_age_mean)
print('trigger_above_60_age_mean = ', trigger_above_60_age_mean)

In [ ]:
datapool_ESRD_age = datapool_ESRD.groupby('patient_sk').agg({'Age': lambda x: (x.iloc[-1] + x.iloc[0])/2}) #getting median of reported ages
np.mean(datapool_ESRD_age.Age)

datapool_ESRD_age = data_no_trigger.groupby('patient_sk').agg({'Age': lambda x: (x.iloc[-1] + x.iloc[0])/2}) #getting median of reported ages
np.mean(datapool_ESRD_age.Age)

In [ ]:
#Gender

trigger_below_20_Female_count = 0
trigger_between_20_40_Female_count = 0
trigger_between_40_60_Female_count = 0
trigger_above_60_Female_count = 0

trigger_below_20_Male_count = 0
trigger_between_20_40_Male_count = 0
trigger_between_40_60_Male_count = 0
trigger_above_60_Male_count = 0

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        if Genders[count_list.index(patient_sk)] == 'Female':
            trigger_below_20_Female_count = trigger_below_20_Female_count + 1
        elif Genders[count_list.index(patient_sk)] == 'Male':
            trigger_below_20_Male_count = trigger_below_20_Male_count + 1

    elif patient_sk in trigger_between_20_40:
        if Genders[count_list.index(patient_sk)] == 'Female':
            trigger_between_20_40_Female_count = trigger_between_20_40_Female_count + 1
        elif Genders[count_list.index(patient_sk)] == 'Male':
            trigger_between_20_40_Male_count = trigger_between_20_40_Male_count + 1

    elif patient_sk in trigger_between_40_60:
        if Genders[count_list.index(patient_sk)] == 'Female':
            trigger_between_40_60_Female_count = trigger_between_40_60_Female_count + 1
        elif Genders[count_list.index(patient_sk)] == 'Male':
            trigger_between_40_60_Male_count = trigger_between_40_60_Male_count + 1
            
    elif patient_sk in trigger_above_60:
        if Genders[count_list.index(patient_sk)] == 'Female':
            trigger_above_60_Female_count = trigger_above_60_Female_count + 1
        elif Genders[count_list.index(patient_sk)] == 'Male':
            trigger_above_60_Male_count = trigger_above_60_Male_count + 1            
            
print('trigger_below_20_Female_count = ', trigger_below_20_Female_count)
print('trigger_between_20_40_Female_count = ', trigger_between_20_40_Female_count)
print('trigger_between_40_60_Female_count = ', trigger_between_40_60_Female_count)
print('trigger_above_60_Female_count = ', trigger_above_60_Female_count)
print('\n')
print('trigger_below_20_Male_count = ', trigger_below_20_Male_count)
print('trigger_between_20_40_Male_count = ', trigger_between_20_40_Male_count)
print('trigger_between_40_60_Male_count = ', trigger_between_40_60_Male_count)
print('trigger_above_60_Male_count = ', trigger_above_60_Male_count)

In [ ]:
datapool_ESRD_gender_female = data_no_trigger[data_no_trigger.Gender == 'Female']
datapool_ESRD_gender_male = data_no_trigger[data_no_trigger.Gender == 'Male']


datapool_ESRD_gender_female.patient_sk.unique().shape[0]
datapool_ESRD_gender_male.patient_sk.unique().shape[0]

In [ ]:
#Races

trigger_below_20_African_American_count = 0
trigger_between_20_40_African_American_count = 0
trigger_between_40_60_African_American_count = 0
trigger_above_60_African_American_count = 0

trigger_below_20_Native_American_count = 0
trigger_between_20_40_Native_American_count = 0
trigger_between_40_60_Native_American_count = 0
trigger_above_60_Native_American_count = 0

trigger_below_20_Asian_Pacific_Islander_count = 0
trigger_between_20_40_Asian_Pacific_Islander_count = 0
trigger_between_40_60_Asian_Pacific_Islander_count = 0
trigger_above_60_Asian_Pacific_Islander_count = 0

trigger_below_20_Hispanic_count = 0
trigger_between_20_40_Hispanic_count = 0
trigger_between_40_60_Hispanic_count = 0
trigger_above_60_Hispanic_count = 0

trigger_below_20_Biracial_count = 0
trigger_between_20_40_Biracial_count = 0
trigger_between_40_60_Biracial_count = 0
trigger_above_60_Biracial_count = 0

trigger_below_20_Caucasian_count = 0
trigger_between_20_40_Caucasian_count = 0
trigger_between_40_60_Caucasian_count = 0
trigger_above_60_Caucasian_count = 0

trigger_below_20_Mid_Eastern_Indian_count = 0
trigger_between_20_40_Mid_Eastern_Indian_count = 0
trigger_between_40_60_Mid_Eastern_Indian_count = 0
trigger_above_60_Mid_Eastern_Indian_count = 0

trigger_below_20_Unknown_count = 0
trigger_between_20_40_Unknown_count = 0
trigger_between_40_60_Unknown_count = 0
trigger_above_60_Unknown_count = 0

for patient_sk in count_list:
    
    if patient_sk in trigger_below_20:
        if Races[count_list.index(patient_sk)] == 'African American':
            trigger_below_20_African_American_count = trigger_below_20_African_American_count + 1
        elif Races[count_list.index(patient_sk)] == 'Native American':
            trigger_below_20_Native_American_count = trigger_below_20_Native_American_count + 1
        elif Races[count_list.index(patient_sk)] in ['Asian', 'Asian/Pacific Islander', 'Pacific Islander'] :
            trigger_below_20_Asian_Pacific_Islander_count = trigger_below_20_Asian_Pacific_Islander_count + 1
        elif Races[count_list.index(patient_sk)] == 'Hispanic':
            trigger_below_20_Hispanic_count = trigger_below_20_Hispanic_count + 1
        elif Races[count_list.index(patient_sk)] == 'Mid Eastern Indian':
            trigger_below_20_Mid_Eastern_Indian_count = trigger_below_20_Mid_Eastern_Indian_count + 1
        elif Races[count_list.index(patient_sk)] == 'Caucasian':
            trigger_below_20_Caucasian_count = trigger_below_20_Caucasian_count + 1
        elif Races[count_list.index(patient_sk)] ==  'Biracial':
            trigger_below_20_Biracial_count = trigger_below_20_Biracial_count + 1            
        else:
            trigger_below_20_Unknown_count = trigger_below_20_Unknown_count + 1    

    if patient_sk in trigger_between_20_40:
        if Races[count_list.index(patient_sk)] == 'African American':
            trigger_between_20_40_African_American_count = trigger_between_20_40_African_American_count + 1
        elif Races[count_list.index(patient_sk)] == 'Native American':
            trigger_between_20_40_Native_American_count = trigger_between_20_40_Native_American_count + 1
        elif Races[count_list.index(patient_sk)] in ['Asian', 'Asian/Pacific Islander', 'Pacific Islander'] :
            trigger_between_20_40_Asian_Pacific_Islander_count = trigger_between_20_40_Asian_Pacific_Islander_count + 1
        elif Races[count_list.index(patient_sk)] == 'Hispanic':
            trigger_between_20_40_Hispanic_count = trigger_between_20_40_Hispanic_count + 1
        elif Races[count_list.index(patient_sk)] == 'Mid Eastern Indian':
            trigger_between_20_40_Mid_Eastern_Indian_count = trigger_between_20_40_Mid_Eastern_Indian_count + 1
        elif Races[count_list.index(patient_sk)] == 'Caucasian':
            trigger_between_20_40_Caucasian_count = trigger_between_20_40_Caucasian_count + 1
        elif Races[count_list.index(patient_sk)] ==  'Biracial':
            trigger_between_20_40_Biracial_count = trigger_between_20_40_Biracial_count + 1            
        else:
            trigger_between_20_40_Unknown_count = trigger_between_20_40_Unknown_count + 1  
            
    if patient_sk in trigger_between_40_60:
        if Races[count_list.index(patient_sk)] == 'African American':
            trigger_between_40_60_African_American_count = trigger_between_40_60_African_American_count + 1
        elif Races[count_list.index(patient_sk)] == 'Native American':
            trigger_between_40_60_Native_American_count = trigger_between_40_60_Native_American_count + 1
        elif Races[count_list.index(patient_sk)] in ['Asian', 'Asian/Pacific Islander', 'Pacific Islander'] :
            trigger_between_40_60_Asian_Pacific_Islander_count = trigger_between_40_60_Asian_Pacific_Islander_count + 1
        elif Races[count_list.index(patient_sk)] == 'Hispanic':
            trigger_between_40_60_Hispanic_count = trigger_between_40_60_Hispanic_count + 1
        elif Races[count_list.index(patient_sk)] == 'Mid Eastern Indian':
            trigger_between_40_60_Mid_Eastern_Indian_count = trigger_between_40_60_Mid_Eastern_Indian_count + 1
        elif Races[count_list.index(patient_sk)] == 'Caucasian':
            trigger_between_40_60_Caucasian_count = trigger_between_40_60_Caucasian_count + 1
        elif Races[count_list.index(patient_sk)] ==  'Biracial':
            trigger_between_40_60_Biracial_count = trigger_between_40_60_Biracial_count + 1            
        else:
            trigger_between_40_60_Unknown_count = trigger_between_40_60_Unknown_count + 1  
            
    if patient_sk in trigger_above_60:
        if Races[count_list.index(patient_sk)] == 'African American':
            trigger_above_60_African_American_count = trigger_above_60_African_American_count + 1
        elif Races[count_list.index(patient_sk)] == 'Native American':
            trigger_above_60_Native_American_count = trigger_above_60_Native_American_count + 1
        elif Races[count_list.index(patient_sk)] in ['Asian', 'Asian/Pacific Islander', 'Pacific Islander'] :
            trigger_above_60_Asian_Pacific_Islander_count = trigger_above_60_Asian_Pacific_Islander_count + 1
        elif Races[count_list.index(patient_sk)] == 'Hispanic':
            trigger_above_60_Hispanic_count = trigger_above_60_Hispanic_count + 1
        elif Races[count_list.index(patient_sk)] == 'Mid Eastern Indian':
            trigger_above_60_Mid_Eastern_Indian_count = trigger_above_60_Mid_Eastern_Indian_count + 1
        elif Races[count_list.index(patient_sk)] == 'Caucasian':
            trigger_above_60_Caucasian_count = trigger_above_60_Caucasian_count + 1
        elif Races[count_list.index(patient_sk)] ==  'Biracial':
            trigger_above_60_Biracial_count = trigger_above_60_Biracial_count + 1            
        else:
            trigger_above_60_Unknown_count = trigger_above_60_Unknown_count + 1    
            

print('trigger_below_20_African_American_count = ', trigger_below_20_African_American_count/456)
print('trigger_between_20_40_African_American_count = ', trigger_between_20_40_African_American_count/962)
print('trigger_between_40_60_African_American_count = ', trigger_between_40_60_African_American_count/2379)
print('trigger_above_60_African_American_count = ', trigger_above_60_African_American_count/1124)
print('\n')
print('trigger_below_20_Native_American_count = ', trigger_below_20_Native_American_count/456)
print('trigger_between_20_40_Native_American_count = ', trigger_between_20_40_Native_American_count/962)
print('trigger_between_40_60_Native_American_count = ', trigger_between_40_60_Native_American_count/2379)
print('trigger_above_60_Native_American_count = ', trigger_above_60_Native_American_count/1124)
print('\n')
print('trigger_below_20_Asian_Pacific_Islander_count = ', trigger_below_20_Asian_Pacific_Islander_count/456)
print('trigger_between_20_40_Asian_Pacific_Islander_count = ', trigger_between_20_40_Asian_Pacific_Islander_count/962)
print('trigger_between_40_60_Asian_Pacific_Islander_count = ', trigger_between_40_60_Asian_Pacific_Islander_count/2379)
print('trigger_above_60_Asian_Pacific_Islander_count = ', trigger_above_60_Asian_Pacific_Islander_count/1124)
print('\n')
print('trigger_below_20_Hispanic_count = ', trigger_below_20_Hispanic_count/456)
print('trigger_between_20_40_Hispanic_count = ', trigger_between_20_40_Hispanic_count/962)
print('trigger_between_40_60_Hispanic_count = ', trigger_between_40_60_Hispanic_count/2379)
print('trigger_above_60_Hispanic_count = ', trigger_above_60_Hispanic_count/1124)
print('\n')
print('trigger_below_20_Mid_Eastern_Indian_count = ', trigger_below_20_Mid_Eastern_Indian_count/456)
print('trigger_between_20_40_Mid_Eastern_Indian_count = ', trigger_between_20_40_Mid_Eastern_Indian_count/962)
print('trigger_between_40_60_Mid_Eastern_Indian_count = ', trigger_between_40_60_Mid_Eastern_Indian_count/2379)
print('trigger_above_60_Mid_Eastern_Indian_count = ', trigger_above_60_Mid_Eastern_Indian_count/1124)
print('\n')
print('trigger_below_20_Caucasian_count = ', trigger_below_20_Caucasian_count/456)
print('trigger_between_20_40_Caucasian_count = ', trigger_between_20_40_Caucasian_count/962)
print('trigger_between_40_60_Caucasian_count = ', trigger_between_40_60_Caucasian_count/2379)
print('trigger_above_60_Caucasian_count = ', trigger_above_60_Caucasian_count/1124)
print('\n')
print('trigger_below_20_Biracial_count = ', trigger_below_20_Biracial_count/456)
print('trigger_between_20_40_Biracial_count = ', trigger_between_20_40_Biracial_count/962)
print('trigger_between_40_60_Biracial_count = ', trigger_between_40_60_Biracial_count/2379)
print('trigger_above_60_Biracial_count = ', trigger_above_60_Biracial_count/1124)
print('\n')
print('trigger_below_20_Unknown_count = ', trigger_below_20_Unknown_count/456)
print('trigger_between_20_40_Unknown_count = ', trigger_between_20_40_Unknown_count/962)
print('trigger_between_40_60_Unknown_count = ', trigger_between_40_60_Unknown_count/2379)
print('trigger_above_60_Unknown_count = ', trigger_above_60_Unknown_count/1124)

In [ ]:
datapool_ESRD_race = datapool_ESRD.groupby('patient_sk').agg({'Race': lambda x: x.iloc[-1]}) #getting the last non_NONE race reported
dataaa_ESRD = datapool_ESRD_race.reset_index()

African_Americans = dataaa_ESRD[dataaa_ESRD['Race'] == 'African American']['Race']
Native_Americans = dataaa_ESRD[dataaa_ESRD['Race'] == 'Native American']['Race']
Asian = dataaa_ESRD[dataaa_ESRD['Race'] == 'Asian']['Race']
Pacific = dataaa_ESRD[dataaa_ESRD['Race'] == 'Pacific Islander']['Race']
Hispanic = dataaa_ESRD[dataaa_ESRD['Race'] == 'Hispanic']['Race']
Mid_Eastern_Indian = dataaa_ESRD[dataaa_ESRD['Race'] == 'Mid Eastern Indian']['Race']
Caucasian = dataaa_ESRD[dataaa_ESRD['Race'] == 'Caucasian']['Race']
Biracial = dataaa_ESRD[dataaa_ESRD['Race'] == 'Biracial']['Race']

print('African_Americans', len(African_Americans), (len(African_Americans)/len(dataaa_ESRD))*100)
print('Native_Americans', len(Native_Americans), (len(Native_Americans)/len(dataaa_ESRD))*100)
print('Asian', len(Asian), (len(Asian)/len(dataaa_ESRD))*100)
print('Pacific', len(Pacific), (len(Pacific)/len(dataaa_ESRD))*100)
print('Hispanic', len(Hispanic), (len(Hispanic)/len(dataaa_ESRD))*100)
print('Mid_Eastern_Indian', len(Mid_Eastern_Indian), (len(Mid_Eastern_Indian)/len(dataaa_ESRD))*100)
print('Caucasian', len(Caucasian), (len(Caucasian)/len(dataaa_ESRD))*100)
print('Biracial', len(Biracial), (len(Biracial)/len(dataaa_ESRD))*100)
print(len(African_Americans) + len(Native_Americans) + len(Asian)+len(Pacific)+len(Hispanic)+len(Mid_Eastern_Indian)+len(Caucasian)+len(Biracial))

In [ ]:
datapool_ESRD_race = data_no_trigger.groupby('patient_sk').agg({'Race': lambda x: x.iloc[-1]}) #getting the last non_NONE race reported
dataaa_ESRD = datapool_ESRD_race.reset_index()

African_Americans = dataaa_ESRD[dataaa_ESRD['Race'] == 'African American']['Race']
Native_Americans = dataaa_ESRD[dataaa_ESRD['Race'] == 'Native American']['Race']
Asian = dataaa_ESRD[dataaa_ESRD['Race'] == 'Asian']['Race']
Pacific = dataaa_ESRD[dataaa_ESRD['Race'] == 'Pacific Islander']['Race']
Hispanic = dataaa_ESRD[dataaa_ESRD['Race'] == 'Hispanic']['Race']
Mid_Eastern_Indian = dataaa_ESRD[dataaa_ESRD['Race'] == 'Mid Eastern Indian']['Race']
Caucasian = dataaa_ESRD[dataaa_ESRD['Race'] == 'Caucasian']['Race']
Biracial = dataaa_ESRD[dataaa_ESRD['Race'] == 'Biracial']['Race']

print('African_Americans', len(African_Americans), (len(African_Americans)/len(dataaa_ESRD))*100)
print('Native_Americans', len(Native_Americans), (len(Native_Americans)/len(dataaa_ESRD))*100)
print('Asian', len(Asian), (len(Asian)/len(dataaa_ESRD))*100)
print('Pacific', len(Pacific), (len(Pacific)/len(dataaa_ESRD))*100)
print('Hispanic', len(Hispanic), (len(Hispanic)/len(dataaa_ESRD))*100)
print('Mid_Eastern_Indian', len(Mid_Eastern_Indian), (len(Mid_Eastern_Indian)/len(dataaa_ESRD))*100)
print('Caucasian', len(Caucasian), (len(Caucasian)/len(dataaa_ESRD))*100)
print('Biracial', len(Biracial), (len(Biracial)/len(dataaa_ESRD))*100)
print(len(African_Americans) + len(Native_Americans) + len(Asian)+len(Pacific)+len(Hispanic)+len(Mid_Eastern_Indian)+len(Caucasian)+len(Biracial))

## Medications 

In [ ]:
merged_dataset['Diagnosis_admission_date_Nicotine_Tobacco_dependency'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_Nicotine_Tobacco_dependency'], errors='coerce')
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')

Diagnosis_admission_date_Nicotine_Tobacco_dependency = list(merged_dataset['Diagnosis_admission_date_Nicotine_Tobacco_dependency'])
Trigger_date = list(merged_dataset['Trigger_date'])

for i in range(len(Diagnosis_admission_date_Nicotine_Tobacco_dependency)):
    if pd.isnull(Diagnosis_admission_date_Nicotine_Tobacco_dependency[i]):
        Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] = pd.Timestamp('1800-01-01')

count = 0
count_list = []
for i in range(len(Diagnosis_admission_date_Nicotine_Tobacco_dependency)):
    if Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] <= Trigger_date[i] and Diagnosis_admission_date_Nicotine_Tobacco_dependency[i] != pd.Timestamp('1800-01-01'):
        count = count + 1
        count_list.append(merged_dataset['patient_sk'][i])
        
print('Diagnosis_admission_date_Nicotine_Tobacco_dependency: ',count)

# Now, the categorization
print('trigger_below_20: ',len(list(set(count_list) & set(trigger_below_20))))
print('trigger_between_20_40: ',len(list(set(count_list) & set(trigger_between_20_40))))
print('trigger_between_40_60: ',len(list(set(count_list) & set(trigger_between_40_60))))
print('trigger_above_60: ',len(list(set(count_list) & set(trigger_above_60))))

# Now, the Normal group

In [ ]:
datapool_control = pd.read_csv("Final_Normal_group_done_pandas.csv")
datapool_control = datapool_control.drop(columns=datapool_control.columns[0])

datapool_control = datapool_control.drop(datapool_control.index[np.isinf(datapool_control.eGFR_EPI) == True], axis = 0)
datapool_control = datapool_control.drop_duplicates()

#some patients have less than 9 datapoinsts!!! AFTER DROPPING DUPLICATES
datapool_control_dropped = datapool_control.groupby('patient_sk').agg({'eGFR_EPI' : 'count'}).reset_index()[datapool_control.groupby('patient_sk').agg({'eGFR_EPI' : 'count'}).reset_index().eGFR_EPI >=9]
datapool_control_dropped = datapool_control_dropped.drop('eGFR_EPI', axis =1)
datapool_control = datapool_control.merge(datapool_control_dropped, on = 'patient_sk', how = 'inner')
#Pulling out each patient's data 
#Also. sortinh the data by cSr lavel measurement data and reindexing it

datapool_ESRD['Date'] = pd.to_datetime(datapool_ESRD['Date'])
datapool_ESRD['Date'] = pd.to_datetime(datapool_ESRD['Date'])
datapool_ESRD_dates = datapool_ESRD.groupby('patient_sk').agg({'Date': 'min'})
datapool_ESRD_dates = datapool_ESRD_dates.reset_index()
datapool_ESRD = datapool_ESRD.merge(datapool_ESRD_dates, on = 'patient_sk', how='left')
datapool_ESRD['Date_seconds'] = datapool_ESRD['Date_x'] - datapool_ESRD['Date_y']
datapool_ESRD = datapool_ESRD.rename({'Date_x':'Date'}, axis = 1)
datapool_ESRD = datapool_ESRD.drop('Date_y', axis = 1)
datapool_ESRD['Date_seconds'] = datapool_ESRD['Date_seconds'].dt.total_seconds()

#Getting rid of Normal min eGFR < 60

datapool_control_patients = datapool_control.groupby('patient_sk').agg({'eGFR_EPI': 'min'})
datapool_control_patients = datapool_control_patients[datapool_control_patients['eGFR_EPI']>=60]
datapool_control_patients = datapool_control_patients.reset_index()

datapool_control = datapool_control_patients.merge(datapool_control, on = 'patient_sk', how = 'inner')
datapool_control = datapool_control.rename({'eGFR_EPI_y':'eGFR_EPI'}, axis = 1)
datapool_control = datapool_control.drop('eGFR_EPI_x', axis = 1)

patients_list_control_above_60 = list(set(np.unique(list(datapool_control['patient_sk']))))

patients_list_Normal = patients_list_control_above_60

patients_list_Normal = list(set(np.unique(list(datapool_control['patient_sk']))))
len(patients_list_Normal)

In [ ]:
patients_list_Normal = datapool_control_patients.merge(datapool_Normal, on = 'patient_sk', how = 'inner')

In [ ]:
datapool_control_age = datapool_control.groupby('patient_sk').agg({'Age': lambda x: (x.iloc[-1] + x.iloc[0])/2}) #getting median of reported ages
datapool_control_age = datapool_control_age.reset_index()
np.mean(datapool_control_age.Age)

In [ ]:
datapool_ESRD_age = datapool_ESRD.groupby('patient_sk').agg({'Age': lambda x: (x.iloc[-1] + x.iloc[0])/2}) #getting median of reported ages
datapool_ESRD_age = datapool_ESRD_age.reset_index()
np.mean(datapool_ESRD_age.Age)

In [ ]:
patients_list_Normal_gender = patients_list_Normal[patients_list_Normal['Gender']<'Not Mapped']
patients_list_Normal_gender = patients_list_Normal_gender.drop_duplicates('patient_sk')
Genders_female = patients_list_Normal_gender[patients_list_Normal_gender['Gender'] == 'Female']['Gender']
Genders_male = patients_list_Normal_gender[patients_list_Normal_gender['Gender'] == 'Male']['Gender']
print('Genders_female', len(list(Genders_female)), (len(list(Genders_female))/len(patients_list_Normal_gender))*100)
print('Genders_male', len(list(Genders_male)), (len(list(Genders_male))/len(patients_list_Normal_gender))*100)

In [ ]:
dataaa_Normal = patients_list_Normal.drop_duplicates('patient_sk')

In [ ]:
dataaa_Normal['Race'].unique()

In [ ]:
dataaa_Normal = patients_list_Normal.drop_duplicates('patient_sk')dataaa_Normal

In [ ]:
datapool_Normal_race = datapool_control.groupby('patient_sk').agg({'Race': lambda x: x.iloc[-1]}) #getting the last non_NONE race reported
dataaa_Normal = datapool_Normal_race.reset_index()

African_Americans = dataaa_Normal[dataaa_Normal['Race'] == 'African American']['Race']
Native_Americans = dataaa_Normal[dataaa_Normal['Race'] == 'Native American']['Race']
Asian = dataaa_Normal[dataaa_Normal['Race'] == 'Asian']['Race']
Pacific = dataaa_Normal[dataaa_Normal['Race'] == 'Pacific Islander']['Race']
Hispanic = dataaa_Normal[dataaa_Normal['Race'] == 'Hispanic']['Race']
Mid_Eastern_Indian = dataaa_Normal[dataaa_Normal['Race'] == 'Mid Eastern Indian']['Race']
Caucasian = dataaa_Normal[dataaa_Normal['Race'] == 'Caucasian']['Race']
Biracial = dataaa_Normal[dataaa_Normal['Race'] == 'Biracial']['Race']

print('African_Americans', len(African_Americans), (len(African_Americans)/len(dataaa_Normal))*100)
print('Native_Americans', len(Native_Americans), (len(Native_Americans)/len(dataaa_Normal))*100)
print('Asian', len(Asian), (len(Asian)/len(dataaa_Normal))*100)
print('Pacific', len(Pacific), (len(Pacific)/len(dataaa_Normal))*100)
print('Hispanic', len(Hispanic), (len(Hispanic)/len(dataaa_Normal))*100)
print('Mid_Eastern_Indian', len(Mid_Eastern_Indian), (len(Mid_Eastern_Indian)/len(dataaa_Normal))*100)
print('Caucasian', len(Caucasian), (len(Caucasian)/len(dataaa_Normal))*100)
print('Biracial', len(Biracial), (len(Biracial)/len(dataaa_Normal))*100)
print(len(African_Americans) + len(Native_Americans) + len(Asian)+len(Pacific)+len(Hispanic)+len(Mid_Eastern_Indian)+len(Caucasian)+len(Biracial))

In [ ]:
patients_list_Normal_race = patients_list_Normal[patients_list_Normal['Race']<'Not Mapped']
patients_list_Normal_race = patients_list_Normal_race.drop_duplicates('patient_sk')

dataaa_Normal = patients_list_Normal_race

African_Americans = dataaa_Normal[dataaa_Normal['Race'] == 'African American']['Race']
Native_Americans = dataaa_Normal[dataaa_Normal['Race'] == 'Native American']['Race']
Asian = dataaa_Normal[dataaa_Normal['Race'] == 'Asian']['Race']
Pacific = dataaa_Normal[dataaa_Normal['Race'] == 'Pacific Islander']['Race']
Hispanic = dataaa_Normal[dataaa_Normal['Race'] == 'Hispanic']['Race']
Mid_Eastern_Indian = dataaa_Normal[dataaa_Normal['Race'] == 'Mid Eastern Indian']['Race']
Caucasian = dataaa_Normal[dataaa_Normal['Race'] == 'Caucasian']['Race']
Biracial = dataaa_Normal[dataaa_Normal['Race'] == 'Biracial']['Race']
Other = dataaa_Normal[dataaa_Normal['Race'] == 'Other']['Race']

print('African_Americans', len(African_Americans), (len(African_Americans)/len(dataaa_Normal))*100)
print('Native_Americans', len(Native_Americans), (len(Native_Americans)/len(dataaa_Normal))*100)
print('Asian', len(Asian), (len(Asian)/len(dataaa_Normal))*100)
print('Pacific', len(Pacific), (len(Pacific)/len(dataaa_Normal))*100)
print('Hispanic', len(Hispanic), (len(Hispanic)/len(dataaa_Normal))*100)
print('Mid_Eastern_Indian', len(Mid_Eastern_Indian), (len(Mid_Eastern_Indian)/len(dataaa_Normal))*100)
print('Caucasian', len(Caucasian), (len(Caucasian)/len(dataaa_Normal))*100)
print('Biracial', len(Biracial), (len(Biracial)/len(dataaa_Normal))*100)
print('Other', len(Other), (len(Other)/len(dataaa_Normal))*100)

print(len(African_Americans) + len(Native_Americans) + len(Asian)+len(Pacific)+len(Hispanic)+len(Mid_Eastern_Indian)+len(Caucasian)+len(Biracial))


In [ ]:
patients_list_Normal_race = patients_list_Normal[patients_list_Normal['Race']>='Not Mapped']
patients_list_Normal_race = patients_list_Normal_race.drop_duplicates('patient_sk')

dataaa_Normal = patients_list_Normal_race

African_Americans = dataaa_Normal[dataaa_Normal['Race'] == 'African American']['Race']
Native_Americans = dataaa_Normal[dataaa_Normal['Race'] == 'Native American']['Race']
Asian = dataaa_Normal[dataaa_Normal['Race'] == 'Asian']['Race']
Pacific = dataaa_Normal[dataaa_Normal['Race'] == 'Pacific Islander']['Race']
Hispanic = dataaa_Normal[dataaa_Normal['Race'] == 'Hispanic']['Race']
Mid_Eastern_Indian = dataaa_Normal[dataaa_Normal['Race'] == 'Mid Eastern Indian']['Race']
Caucasian = dataaa_Normal[dataaa_Normal['Race'] == 'Caucasian']['Race']
Biracial = dataaa_Normal[dataaa_Normal['Race'] == 'Biracial']['Race']
Other = dataaa_Normal[dataaa_Normal['Race'] == 'Other']['Race']
Unknown = dataaa_Normal[dataaa_Normal['Race'] == 'Unknown']['Race']

print('African_Americans', len(African_Americans), (len(African_Americans)/len(dataaa_Normal))*100)
print('Native_Americans', len(Native_Americans), (len(Native_Americans)/len(dataaa_Normal))*100)
print('Asian', len(Asian), (len(Asian)/len(dataaa_Normal))*100)
print('Pacific', len(Pacific), (len(Pacific)/len(dataaa_Normal))*100)
print('Hispanic', len(Hispanic), (len(Hispanic)/len(dataaa_Normal))*100)
print('Mid_Eastern_Indian', len(Mid_Eastern_Indian), (len(Mid_Eastern_Indian)/len(dataaa_Normal))*100)
print('Caucasian', len(Caucasian), (len(Caucasian)/len(dataaa_Normal))*100)
print('Biracial', len(Biracial), (len(Biracial)/len(dataaa_Normal))*100)
print('Other', len(Other), (len(Other)/len(dataaa_Normal))*100)
print('Unknown', len(Unknown), (len(Unknown)/len(dataaa_Normal))*100)
print(len(African_Americans) + len(Native_Americans) + len(Asian)+len(Pacific)+len(Hispanic)+len(Mid_Eastern_Indian)+len(Caucasian)+len(Biracial))


### For further information please contact rzz5164@psu.edu